# Импорты

In [1]:
from logging import disable

import numpy as np
import pandas as pd
import scipy.stats as ss
import matplotlib.pyplot as plt
import statsmodels.stats.api as sms
import seaborn as sns
from statsmodels.stats.proportion import proportion_confint

# Анализ

In [2]:
df = pd.read_csv("proxy_metrics_homework_3.csv")

df.head()

,user_id,views,contacts,last_7_views,last_3_views,last_7_contacts,last_3_contacts,live_days,messages_events,max_views_delta,add_to_favor_events,is_bought
0,151042503,84,2,84,39,2,2,8,0,6,0.0,0
1,151212422,95,0,95,36,0,0,8,0,7,0.0,0
2,151206815,118,3,118,40,3,2,8,0,2,0.0,0
3,151284254,11,0,11,2,0,0,7,0,8,0.0,0
4,151088604,48,2,48,8,2,0,8,0,1,0.0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1193 entries, 0 to 1192
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              1193 non-null   int64  
 1   views                1193 non-null   int64  
 2   contacts             1193 non-null   int64  
 3   last_7_views         1193 non-null   int64  
 4   last_3_views         1193 non-null   int64  
 5   last_7_contacts      1193 non-null   int64  
 6   last_3_contacts      1193 non-null   int64  
 7   live_days            1193 non-null   int64  
 8   messages_events      1193 non-null   int64  
 9   max_views_delta      1193 non-null   int64  
 10  add_to_favor_events  1193 non-null   float64
 11  is_bought            1193 non-null   int64  
dtypes: float64(1), int64(11)
memory usage: 112.0 KB


In [5]:
print(df.is_bought.mean())

0.3403185247275775


In [6]:
df.groupby('is_bought').agg(avg_contacts=('contacts', 'mean')).reset_index()

,is_bought,avg_contacts
0,0,12.978399
1,1,24.325123


In [7]:
df_gr = df.groupby('is_bought').agg(avg_add_to_favor=('add_to_favor_events', 'mean')).reset_index()
display(df_gr)

print(df_gr['avg_add_to_favor'][1] / df_gr['avg_add_to_favor'][0])

,is_bought,avg_add_to_favor
0,0,0.158831
1,1,0.322660


2.0314679802955666


In [10]:
df_gr_0 = df.loc[df.contacts == 0].groupby('is_bought').agg(cnt_users_0=('user_id', 'nunique')).reset_index()
df_gr = df.groupby('is_bought').agg(cnt_users_all=('user_id', 'nunique')).reset_index()

df_gr = df_gr.merge(df_gr_0, on='is_bought')
df_gr['share_0'] = df_gr['cnt_users_0'] / df_gr['cnt_users_all']

display(df_gr)

,is_bought,cnt_users_all,cnt_users_0,share_0
0,0,787,81,0.102922
1,1,406,15,0.036946


In [11]:
df.groupby('is_bought')['live_days'].mean().reset_index()

,is_bought,live_days
0,0,12.722999
1,1,10.219212


In [13]:
df_gr = df.groupby(['is_bought', 'user_id']).agg(cnt_views=('views', 'sum'),
                                    cnt_contacts=('contacts', 'sum')).reset_index()

df_gr['share'] = df_gr['cnt_contacts'] / df_gr['cnt_views']

for group in df_gr.is_bought.unique():
    print(f"{group} - {df_gr[df_gr['is_bought']==group]['share'].mean()}")

0 - 0.022825099781612665
1 - 0.03421006944101362
